# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray "\Resources\establishments.json"` 


In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'travel_db']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

[]


In [6]:
# review the collections in our new database
print(db.list_collection_names())

[]


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

None


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
query = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(query)

InsertOneResult(ObjectId('667a14dd4ca7aae0d55b3489'), acknowledged=True)

In [11]:
# Check that the new restaurant was inserted
new_restaurant = establishments.find_one({"BusinessName":"Penang Flavours"})
new_restaurant


{'_id': ObjectId('667a14dd4ca7aae0d55b3489'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

# Select only the BusinessTypeID and BusinessType fields from the establishments collection
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}

# Capture the results to a variable
results = establishments.find(query, fields)

# Pretty print the results
pprint(results[0])



{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 '_id': ObjectId('667a14dd4ca7aae0d55b3489')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
filter = { "BusinessName":"Penang Flavours" }
 
# Values to be updated.
newvalues = { "$set": { 'BusinessTypeID': 1 } }
 
# Using update_one() method for single update.
establishments.update_one(filter, newvalues)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
new_restaurant = establishments.find_one({"BusinessName":"Penang Flavours"})
new_restaurant

{'_id': ObjectId('667a14dd4ca7aae0d55b3489'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover = {'LocalAuthorityName': 'Dover'}

# Capture the results to a variable
results = establishments.count_documents(dover)

# print the results
pprint(results)

0


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many(dover)
print(result.deleted_count)

0


In [17]:
# Check if any remaining documents include Dover

results = establishments.find_one(dover)
pprint(results)


None


In [18]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('667a14dd4ca7aae0d55b3489'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}} ])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [20]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}} ])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Check that the coordinates are now numbers
pprint(establishments.find_one())

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('667a14dd4ca7aae0d55b3489'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}
